# Framework CLAHE-MCDM: Ejemplo de Uso

Este notebook demuestra el flujo completo del framework para mejora de ortopantomografías:

1. Carga de imagen
2. Aplicación de CLAHE con diferentes parámetros
3. Cálculo de métricas de calidad
4. Optimización multiobjetivo con SMPSO
5. Aplicación de métodos MCDM
6. Visualización de resultados

In [ ]:
# Importaciones necesarias
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')

from clahe.processor import CLAHEProcessor
from metrics.entropy import calculate_entropy
from metrics.ssim import calculate_ssim
from metrics.vqi import calculate_vqi
from optimization.smpso import SMPSO
from optimization.pareto import build_pareto_front, visualize_pareto_front_3d
from mcdm.smarter import SMARTER
from mcdm.topsis import TOPSIS
from mcdm.vikor import VIKOR
from utils.visualization import visualize_clahe_comparison, visualize_mcdm_rankings

## 1. Cargar o Generar Imagen de Ejemplo

Para este ejemplo, generaremos una imagen sintética que simula una ortopantomografía.

In [ ]:
# Generar imagen sintética de ejemplo
np.random.seed(42)
height, width = 400, 800

# Base de imagen con bajo contraste
image = np.ones((height, width), dtype=np.uint8) * 100

# Agregar estructuras simuladas
# Dientes simulados
for i in range(10, 790, 80):
    x_center, y_center = i, 200
    for y in range(height):
        for x in range(width):
            dist = np.sqrt((x - x_center)**2 + (y - y_center)**2)
            if dist < 30:
                image[y, x] = int(150 + 20 * (1 - dist/30))

# Agregar ruido
noise = np.random.normal(0, 10, image.shape)
image = np.clip(image + noise, 0, 255).astype(np.uint8)

plt.figure(figsize=(12, 4))
plt.imshow(image, cmap='gray')
plt.title('Imagen Original Sintética')
plt.axis('off')
plt.show()

print(f"Forma de la imagen: {image.shape}")
print(f"Rango de intensidades: [{image.min()}, {image.max()}]")

## 2. Aplicar CLAHE con Diferentes Parámetros

Probemos CLAHE con varios conjuntos de parámetros para ver su efecto.

In [ ]:
# Configuraciones de CLAHE a probar
configs = [
    {'rx': 4, 'ry': 4, 'clip_limit': 2.0},
    {'rx': 8, 'ry': 8, 'clip_limit': 2.0},
    {'rx': 8, 'ry': 8, 'clip_limit': 3.5},
]

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

for idx, config in enumerate(configs):
    processor = CLAHEProcessor(**config)
    enhanced = processor.process(image)
    
    # Mostrar imagen
    ax_img = axes[0, idx]
    ax_img.imshow(enhanced, cmap='gray')
    ax_img.set_title(f"Rx={config['rx']}, Ry={config['ry']}, C={config['clip_limit']}")
    ax_img.axis('off')
    
    # Mostrar histograma
    ax_hist = axes[1, idx]
    ax_hist.hist(enhanced.ravel(), bins=256, range=(0, 256), alpha=0.7)
    ax_hist.set_xlabel('Intensidad')
    ax_hist.set_ylabel('Frecuencia')
    ax_hist.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Calcular Métricas de Calidad

Evaluemos las imágenes mejoradas con las tres métricas: Entropía, SSIM y VQI.

In [ ]:
import pandas as pd

results = []

for config in configs:
    processor = CLAHEProcessor(**config)
    enhanced = processor.process(image)
    
    entropy = calculate_entropy(enhanced)
    ssim = calculate_ssim(image, enhanced)
    vqi = calculate_vqi(enhanced, reference_image=image)
    
    results.append({
        'Rx': config['rx'],
        'Ry': config['ry'],
        'Clip Limit': config['clip_limit'],
        'Entropía': f"{entropy:.4f}",
        'SSIM': f"{ssim:.4f}",
        'VQI': f"{vqi:.2f}"
    })

df = pd.DataFrame(results)
print("\nMétricas de Calidad para Diferentes Configuraciones:")
print(df.to_string(index=False))

## 4. Optimización Multiobjetivo con SMPSO

Ahora ejecutemos SMPSO para encontrar el Frente de Pareto óptimo.

In [ ]:
# Definir función objetivo
def objective_function(params):
    rx, ry, clip_limit = params
    rx, ry = int(rx), int(ry)
    
    processor = CLAHEProcessor(rx, ry, clip_limit)
    enhanced = processor.process(image)
    
    entropy = calculate_entropy(enhanced)
    ssim = calculate_ssim(image, enhanced)
    vqi = calculate_vqi(enhanced, reference_image=image)
    
    # SMPSO minimiza, pero queremos maximizar
    # Negamos para convertir a minimización
    return [-entropy, -ssim, -vqi]

# Configurar y ejecutar SMPSO
print("Ejecutando SMPSO...")
print("(Esto puede tomar varios minutos)\n")

optimizer = SMPSO(
    n_particles=20,  # Reducido para el ejemplo
    n_iterations=30,  # Reducido para el ejemplo
    bounds=[(2, 16), (2, 16), (1.0, 4.0)],
    verbose=True
)

solutions = optimizer.optimize(objective_function)

# Convertir de vuelta a maximización
for sol in solutions:
    sol['objectives'] = -sol['objectives']

print(f"\nFrente de Pareto: {len(solutions)} soluciones encontradas")

## 5. Visualizar Frente de Pareto 3D

In [ ]:
visualize_pareto_front_3d(
    solutions,
    objective_names=['Entropía', 'SSIM', 'VQI'],
    title='Frente de Pareto 3D - Optimización CLAHE'
)

## 6. Aplicar Métodos MCDM

Apliquemos tres métodos MCDM para seleccionar la mejor solución del Frente de Pareto.

In [ ]:
# Construir matriz de decisión
decision_matrix = np.array([sol['objectives'] for sol in solutions])
weights = np.array([0.33, 0.33, 0.34])

# Aplicar métodos MCDM
methods = {
    'SMARTER': SMARTER(),
    'TOPSIS': TOPSIS(),
    'VIKOR': VIKOR()
}

mcdm_results = {}
mcdm_rankings = {}

for name, method in methods.items():
    best_idx, rankings = method.select(
        decision_matrix,
        weights=weights,
        criteria_types=['benefit', 'benefit', 'benefit']
    )
    
    mcdm_results[name] = best_idx
    mcdm_rankings[name] = rankings
    
    best_sol = solutions[best_idx]
    print(f"\n{name}:")
    print(f"  Mejor solución: #{best_idx}")
    print(f"  Parámetros: Rx={int(best_sol['position'][0])}, "
          f"Ry={int(best_sol['position'][1])}, C={best_sol['position'][2]:.2f}")
    print(f"  Métricas: Entropía={best_sol['objectives'][0]:.4f}, "
          f"SSIM={best_sol['objectives'][1]:.4f}, VQI={best_sol['objectives'][2]:.2f}")

## 7. Visualizar Rankings de MCDM

In [ ]:
visualize_mcdm_rankings(
    mcdm_rankings,
    solutions,
    title='Rankings de Métodos MCDM'
)

## 8. Comparar Imagen Original vs. Mejor Solución

Finalmente, comparemos la imagen original con la mejor solución seleccionada por TOPSIS.

In [ ]:
# Obtener mejor solución según TOPSIS
best_idx_topsis = mcdm_results['TOPSIS']
best_params = solutions[best_idx_topsis]['position']

# Aplicar CLAHE con los parámetros óptimos
optimal_processor = CLAHEProcessor(
    rx=int(best_params[0]),
    ry=int(best_params[1]),
    clip_limit=best_params[2]
)
optimal_enhanced = optimal_processor.process(image)

# Calcular métricas finales
final_metrics = {
    'Entropía': calculate_entropy(optimal_enhanced),
    'SSIM': calculate_ssim(image, optimal_enhanced),
    'VQI': calculate_vqi(optimal_enhanced, reference_image=image)
}

# Visualizar comparación
visualize_clahe_comparison(
    image,
    optimal_enhanced,
    metrics=final_metrics,
    title='Comparación: Original vs. Optimizada (TOPSIS)'
)

## 9. Análisis de Concordancia entre Métodos MCDM

In [ ]:
print("\nAnálisis de Concordancia:")
print("="*50)

selections = list(mcdm_results.values())
unique, counts = np.unique(selections, return_counts=True)

print(f"\nSoluciones seleccionadas:")
for sol_idx, count in zip(unique, counts):
    print(f"  Solución #{sol_idx}: {count} método(s)")

if len(unique) == 1:
    print("\n¡Todos los métodos convergen a la misma solución!")
elif np.max(counts) >= 2:
    most_common = unique[np.argmax(counts)]
    print(f"\nLa solución #{most_common} es la más frecuente ({np.max(counts)} métodos)")
else:
    print("\nLos métodos seleccionan soluciones diferentes.")
    print("Considere analizar las diferencias entre estas soluciones.")

## Conclusión

Este notebook ha demostrado el flujo completo del framework:

1. ✅ Procesamiento CLAHE con múltiples configuraciones
2. ✅ Evaluación con métricas objetivas (Entropía, SSIM, VQI)
3. ✅ Optimización multiobjetivo con SMPSO
4. ✅ Generación del Frente de Pareto 3D
5. ✅ Aplicación de métodos MCDM para selección
6. ✅ Visualización de resultados

El framework proporciona una solución robusta y automatizada para la mejora de ortopantomografías, combinando lo mejor de la optimización multiobjetivo y los métodos de decisión multicriterio.